 単語感情極性対応表　http://www.lr.pi.titech.ac.jp/~takamura/pndic_ja.html と<br>
 ライブドアニュースコーパス　https://www.rondhuit.com/download.html#ldcc を使って分析

<br>

# 参考
https://qiita.com/g-k/items/e49f68d7e2fed6e300ea


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/media'
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/media
chABSA-dataset/     ldcc-20140209.tar.gz  media.ipynb  tweets_open.csv
chABSA-dataset.zip  livedoordata/         pn_ja        tweets_open.gsheet
D18-2018.7.24.xlsx  __MACOSX/             test.tsv
dev.tsv             media2.ipynb          text/
kanjokyokusei.txt   media3.ipynb          train.tsv


In [2]:
!pip install jaconv
!pip install transformers
!apt install git make curl xz-utils file
!apt install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8
!pip install mecab-python3==0.996.5

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#バージョン指定
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
xz-utils is already the newest version (5.2.2-1.3).
curl is already the newest version (7.58.0-2ubuntu3.10).
file is already the newest version (1:5.32-2ubuntu0.4).
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Using cached https://files.pythonhosted.org/packages/18/49/b55a839a77189042960bf96490640c44816073f917d489acbc5d79fa5cc3/mecab_python3-0.996.5-cp36-cp36m-manylinu

In [9]:
import pandas as pd
import numpy as np
import os
import glob
import pathlib
import re
import jaconv
import torch


def get_kanjo_score(fname):
    text_list = []
    text_read_list = []
    text_hinshi_list = []
    score_list = [] #label

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.split(':')#言葉:読み:品詞:スコア
            text_list.append(line[0])
            howread = jaconv.hira2kata(line[1])
            text_read_list.append(howread)
            text_hinshi_list.append(line[2])#カタカナにしてリストへ
            score = float(line[3])
            score_list.append(score)

    score_list = torch.Tensor(score_list)

    return  text_list,text_read_list,text_hinshi_list,score_list
kanjo_score_path =  '/content/gdrive/My Drive/Colab Notebooks/media/kanjokyokusei.txt'#todo 学習用ファイルパス
text_list,text_read_list,text_hinshi_list,score_list = get_kanjo_score(kanjo_score_path)
text_read_list[0]

'スグレル'

In [4]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
import numpy as np

#train_x はテキストのリスト、yはテンソルにしてもOK
#get_dataはテキストのリストとyはテンソルを返す
def get_data(fname):
    text_list = []
    ids_list = [] #label
    #i = 0
    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.split('\t')#todo lineからタイトルを取得
            text = line[0]
            text_list.append(text)
            ids = float(line[1])
            ids_list.append(ids)


    ids_list = torch.Tensor(ids_list)

    return  text_list,ids_list

# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/dev.tsv'
train_text_list,train_ids_list = get_data(train_path)


train_text_list[0]

'バラエティ番組で、あるお笑い芸人が、こんなことを言っていた。'

In [30]:
import MeCab
m = MeCab.Tagger('-Ochasen')
#バグが起きないように入れておくやつ

text_len = len(train_text_list)
sentence_score = np.zeros(text_len)#文章のスコアを保存する

for i in range(text_len):
  m.parse('')
  node = m.parseToNode(train_text_list[i])
  while node:
    # surfaceで単語(表層形)，featureで品詞情報がとれる
    features = node.feature.split(',')
    #print(features)
    if len(features) > 7:
      word = features[7]
    else:
       word = ''
    if word in text_read_list:#辞書の言葉があったら
      text_index = text_read_list.index(word)
      sentence_score[i] += score_list[text_index]
      print("文章:",train_text_list[i],"あった単語",word,"そのスコア",score_list[text_index])
    node = node.next


ストリーミング出力は最後の 5000 行に切り捨てられました。
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうしたほうが長く飛ぶのか、安定するのかといったことを気にしてはいけない。紙飛行機に足が生えるのに何の意味があるのかを深く考えてもいけない。とにかく、紙ヒコーキに足！これだけでいいのである。かなりのインパクトがあるだけで、もうネタ的には十分だ。 あった単語 アシ そのスコア tensor(-0.6780)
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうしたほうが長く飛ぶのか、安定するのかといったことを気にしてはいけない。紙飛行機に足が生えるのに何の意味があるのかを深く考えてもいけない。とにかく、紙ヒコーキに足！これだけでいいのである。かなりのインパクトがあるだけで、もうネタ的には十分だ。 あった単語 ガ そのスコア tensor(0.9951)
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうしたほうが長く飛ぶのか、安定するのかといったことを気にしてはいけない。紙飛行機に足が生えるのに何の意味があるのかを深く考えてもいけない。とにかく、紙ヒコーキに足！これだけでいいのである。かなりのインパクトがあるだけで、もうネタ的には十分だ。 あった単語 ハエル そのスコア tensor(0.0327)
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうしたほうが長く飛ぶのか、安定するのかといったことを気にしてはいけない。紙飛行機に足が生えるのに何の意味があるのかを深く考えてもいけない。とにかく、紙ヒコーキに足！これだけでいいのである。かなりのインパクトがあるだけで、もうネタ的には十分だ。 あった単語 ノ そのスコア tensor(-0.2586)
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうしたほうが長く飛ぶのか、安定するのかといったことを気にしてはいけない。紙飛行機に足が生えるのに何の意味があるのかを深く考えてもいけない。とにかく、紙ヒコーキに足！これだけでいいのである。かなりのインパクトがあるだけで、もうネタ的には十分だ。 あった単語 イミ そのスコア tensor(-0.8508)
文章: 「うわっ紙ヒコーキに足が生えてる！！」と話題になっている。こうし

In [31]:
sentence_score

array([ 0.3026    , -1.3470571 , -4.96904612, ..., -1.45303607,
        1.29537725, -4.28101635])

In [39]:
score_mean = sentence_score.mean()
score_mean

-3.133199140478857

In [40]:
sentence_vs_score_list = []
for i in range(text_len):
  if sentence_score[i] > score_mean:
    sentence_vs_score_list.append(str(train_text_list[i]) + ","+str(sentence_score[i]) +",positive"+"\n")
  else:
    sentence_vs_score_list.append(str(train_text_list[i]) + ","+str(sentence_score[i]) +",negative"+"\n")    

# ファイルに保存

w = open("sentence_score.tsv", "w")
w.writelines(sentence_vs_score_list)
w.close()